In [1]:
# Set the seed for reproducibility
import numpy as np
import random
import tensorflow as tf
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seed(42)

In [2]:
import pandas as pd

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Load the dataset
train_data = pd.read_csv('sign_mnist_train.csv')
test_data = pd.read_csv('sign_mnist_test.csv')

# Extract labels and images
X_train = train_data.iloc[:, 1:].values
y_train = train_data.iloc[:, 0].values
X_test = test_data.iloc[:, 1:].values
y_test = test_data.iloc[:, 0].values

In [3]:

# Adjust labels to be in range 0-23 instead of 0-24
y_train = [i if i < 9 else i - 1 for i in y_train]
y_test = [i if i < 9 else i - 1 for i in y_test]

# Convert lists to numpy arrays
y_train = np.array(y_train)
y_test = np.array(y_test)

# Reshape images to 32x32 (since the images consists of 32x32 pixels)
X_train = X_train.reshape(-1, 32, 32, 1)
X_test = X_test.reshape(-1, 32, 32, 1)

# Normalize pixel values
X_train = X_train / 255.0
X_test = X_test / 255.0

# Create a fixed validation set and a test set from the testing data
X_val, X_val, y_val, y_final_test = train_test_split(X_test, y_test, stratify= y_test, test_size=0.5, random_state=42)

# One-hot encode labels
y_train_enc = to_categorical(y_train, num_classes=24)
y_val_enc = to_categorical(y_val, num_classes=24)
y_val_enc = to_categorical(y_final_test, num_classes=24)

In [4]:
# Let's see how big it is
print(X_train.shape)
print(X_test.shape)
n_total = X_train.shape[0]

(27455, 32, 32, 1)
(7172, 32, 32, 1)


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten



# Define the densely connected model
dense_model = Sequential([
    Flatten(input_shape=(32, 32, 1)),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(24, activation='softmax')
])

# Compile the model
set_seed(42)
dense_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history_dense = dense_model.fit(X_train, y_train_enc, epochs=20, validation_data=(X_val, y_val_enc))

Epoch 1/20


c:\Users\linhn\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.2827 - loss: 2.3754 - val_accuracy: 0.5683 - val_loss: 1.3102
Epoch 2/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6973 - loss: 0.9249 - val_accuracy: 0.7192 - val_loss: 0.9155
Epoch 3/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8297 - loss: 0.5159 - val_accuracy: 0.7462 - val_loss: 0.8934
Epoch 4/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9095 - loss: 0.2890 - val_accuracy: 0.7521 - val_loss: 0.8922
Epoch 5/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9410 - loss: 0.1866 - val_accuracy: 0.7513 - val_loss: 0.9831
Epoch 6/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9664 - loss: 0.1147 - val_accuracy: 0.7607 - val_loss: 1.0020
Epoch 7/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9607 - loss: 0.1248 - val_accuracy: 0.7716 - val_loss: 0.9627
Epoch 8/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9728 - loss: 0.0879 - val_accuracy: 0.8006 - val_

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


# Define the CNN model
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(24, activation='softmax')
])

# Compile the model
set_seed(42)
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history_cnn = cnn_model.fit(X_train, y_train_enc, epochs=20, validation_data=(X_val, y_val_enc))


Epoch 1/20


c:\Users\linhn\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5623 - loss: 1.5057 - val_accuracy: 0.8795 - val_loss: 0.4001
Epoch 2/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9948 - loss: 0.0352 - val_accuracy: 0.9038 - val_loss: 0.3952
Epoch 3/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9988 - loss: 0.0076 - val_accuracy: 0.9163 - val_loss: 0.3932
Epoch 4/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 1.0000 - loss: 0.0011 - val_accuracy: 0.9200 - val_loss: 0.3761
Epoch 5/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 1.0000 - loss: 4.0789e-04 - val_accuracy: 0.9211 - val_loss: 0.3821
Epoch 6/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 1.0000 - loss: 2.3329e-04 - val_accuracy: 0.9214 - val_loss: 0.3918
Epoch 7/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 1.0000 - loss: 1.4209e-04 - val_accuracy: 0.9208 - val_loss: 0.4043
Epoch 8/20
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 1.0000 - loss: 8.7786e-05 - val_accurac

In [44]:
from sklearn.metrics import confusion_matrix
def evaluate_model(name, model_name, X_val, y_val_enc, verbose = 2):
    model_eval = model_name.evaluate(X_val, y_val_enc, verbose = verbose)
    print(f"{name} Model - Test Accuracy: {model_eval[1]:.6f}\n")

    # Detailed evaluation of the model
    y_pred = np.argmax(model_name.predict(X_val), axis=1)

    # Compute accuracy per class, skipping index 9 (for J)
    y_true = np.argmax(y_val_enc, axis=1)
    accuracy_per_class = []
    for i in range(24):
        if np.sum(y_true == i) > 0:
            accuracy_per_class.append(np.mean(y_pred[y_true == i] == i))
        else:
            accuracy_per_class.append(np.nan)  # Handle classes with no samples

    # Filter out NaN values to calculate the median accuracy
    valid_accuracies = [acc for acc in accuracy_per_class if not np.isnan(acc)]
    median_accuracy = np.median(valid_accuracies)

    print(f"Unbiased Median Accuracy: {median_accuracy:.6f}")

    # Identify the letter with the highest individual accuracy
    highest_accuracy_class = np.nanargmax(accuracy_per_class)
    if highest_accuracy_class >= 9:  # Adjust for the missing 'J'
        letter_with_highest_accuracy = chr(highest_accuracy_class + ord('A') + 1)
    else:
        letter_with_highest_accuracy = chr(highest_accuracy_class + ord('A'))
    print(f"Letter with Highest Accuracy: {letter_with_highest_accuracy}")

    # Identify the letter with the lowest individual accuracy
    lowest_accuracy_class = np.nanargmin(accuracy_per_class)
    if lowest_accuracy_class >= 9:  # Adjust for the missing 'J'
        letter_with_lowest_accuracy = chr(lowest_accuracy_class + ord('A') + 1)
    else:
        letter_with_lowest_accuracy = chr(lowest_accuracy_class + ord('A'))
    print(f"Letter with Lowest Accuracy: {letter_with_lowest_accuracy}\n")


    # Calculate the confusion matrix
    conf_matrix = confusion_matrix(y_true, y_pred)

    # Set the diagonal elements to zero to exclude correct classifications
    np.fill_diagonal(conf_matrix, 0)

    # Flatten the confusion matrix and find top error indices
    flat_matrix = conf_matrix.flatten()
    top_indices = np.argsort(-flat_matrix)

    # Filter out zero errors
    non_zero_indices = top_indices[flat_matrix[top_indices] > 0]

    # Get top three non-zero errors
    top_3_indices = non_zero_indices[:3]
    top_3_errors = flat_matrix[top_3_indices]
    top_positions = np.unravel_index(top_3_indices, conf_matrix.shape)

    # Adjust indices to skip 'J' and map to the correct letters
    def adjust_for_j(index):
        return chr(index + ord('A') + 1) if index >= 9 else chr(index + ord('A'))
    
    print("Top three errors:")
    for i in range(len(top_3_errors)):
        print(f"-> Predict {adjust_for_j(top_positions[0][i])} as {adjust_for_j(top_positions[1][i])}")

    if len(top_3_errors) != 3:
        print(f"Only {len(top_3_errors)} errors detected\n")

    # # Get the top three most common errors with adjustment for 'J'
    # common_errors = [(adjust_for_j(errors[0][i]), adjust_for_j(errors[1][i])) for i in range(3)]

    # print(f"Most Common Errors: {common_errors}")

    # Report overall mean accuracy and accuracy per letter
    mean_accuracy = np.nanmean(accuracy_per_class)
    print(f"Overall Mean Accuracy: {mean_accuracy:.6f}\n")

    # Print each letter and its accuracy
    letters = [chr(i + ord('A')) for i in range(26) if i not in [9, 25]]
    for i, acc in enumerate(accuracy_per_class):
        print(f"Letter {letters[i]}: Accuracy {acc:.6f}")


Part 2

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the data augmentation generator
datagen = ImageDataGenerator(
    rotation_range=10,        # Randomly rotate images by 10 degrees
    width_shift_range=0.1,    # Randomly translate images horizontally by 10% of the width
    height_shift_range=0.1,   # Randomly translate images vertically by 10% of the height
    zoom_range=0.1,           # Randomly zoom images by 10%
    horizontal_flip=True      # Randomly flip images horizontally
)


In [9]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
# Early stopping and learning rate reduction callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam


def create_dense_model_with_regularization(layer_sizes, learning_rate=0.001, l2_lambda=0.01):
    set_seed(42)
    model = Sequential()
    model.add(Flatten(input_shape=(32, 32, 1)))
    for size in layer_sizes:
        model.add(Dense(size, activation='relu', kernel_regularizer=l2(l2_lambda)))
        model.add(BatchNormalization())
    model.add(Dense(24, activation='softmax', kernel_regularizer=l2(l2_lambda)))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow import keras

# def create_cnn_model_with_regularization(conv_layers, dense_layers, learning_rate=0.1, l2_lambda=0.01):
#     model = Sequential()
#     # model = keras.models.Sequential()
#     for filters, kernel_size in conv_layers:
#         model.add(Conv2D(filters, kernel_size, activation='relu', padding='same', kernel_regularizer=l2(l2_lambda), input_shape=(32, 32, 1)))
#         model.add(BatchNormalization())
#         model.add(MaxPooling2D((2, 2), padding='same'))
#     model.add(Flatten())
#     for size in dense_layers:
#         model.add(Dense(size, activation='relu', kernel_regularizer=l2(l2_lambda)))
#         model.add(BatchNormalization())
#     model.add(Dense(24, activation='softmax', kernel_regularizer=l2(l2_lambda)))
#     model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
#     return model

def create_cnn_model_with_regularization(conv_layers, dense_layers, learning_rate=0.001):
    model = Sequential()
    # model = keras.models.Sequential()
    for filters, kernel_size in conv_layers:
        model.add(Conv2D(filters, kernel_size, activation='relu', input_shape=(32, 32, 1)))
        model.add(BatchNormalization())
        model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    for size in dense_layers:
        model.add(Dense(size, activation='relu'))
        model.add(BatchNormalization())
    model.add(Dense(24, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [12]:
def return_best_model(group_name, model_group, X_train, y_train_enc, batch_size = 30):
    # Train and evaluate each  model
    model_histories = []
    for model in model_group:
        print(f'Model for {group_name}: {model}')
        set_seed(42)
        history = model.fit(
            datagen.flow(X_train, y_train_enc, batch_size=batch_size),
            steps_per_epoch=len(X_train) // batch_size,
            epochs=50,
            validation_data=(X_val, y_val_enc),
            callbacks=[early_stopping, reduce_lr],
            verbose=1
        )
        val_accuracy = history.history['val_accuracy'][-1]
        model_histories.append((val_accuracy, model))

    # Determine the best model based on validation accuracy
    best_val_accuracy, best_model = max(model_histories, key=lambda item: item[0])

    print(f'The best model is: {best_model} with a validation accuracy of: {best_val_accuracy}')
    return model_histories

In [13]:
# Define different dense models to experiment with
dense_models = [
    create_dense_model_with_regularization([512, 256], learning_rate=0.001),
    create_dense_model_with_regularization([1024, 512, 256], learning_rate=0.001),
    create_dense_model_with_regularization([1024, 512, 256, 128], learning_rate=0.001)
]

In [14]:
# Define different CNN models with regularization to experiment with
cnn_models = [
    create_cnn_model_with_regularization([(32, (3, 3)), (64, (3, 3))], [128],learning_rate=0.001),
    create_cnn_model_with_regularization([(32, (3, 3)), (64, (3, 3)), (128, (3, 3))], [256], learning_rate=0.001),
    create_cnn_model_with_regularization([(16, (3,3)), (32, (3, 3)), (64, (3, 3))], [512, 128], learning_rate=0.01)
]

In [15]:
dense_histories = return_best_model("Dense", dense_models, X_train, y_train_enc, 30)

Model for Dense: <Sequential name=sequential_2, built=True>
Epoch 1/50
 33/915 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.1201 - loss: 13.2056

c:\Users\linhn\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


915/915 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.2234 - loss: 6.9141 - val_accuracy: 0.0449 - val_loss: 11.3997 - learning_rate: 0.0010
Epoch 2/50
915/915 ━━━━━━━━━━━━━━━━━━━━ 0s 195us/step - accuracy: 0.2667 - loss: 2.8160 - val_accuracy: 0.0410 - val_loss: 10.8091 - learning_rate: 0.0010
Epoch 3/50


c:\Users\linhn\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


915/915 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.3164 - loss: 2.5573 - val_accuracy: 0.0678 - val_loss: 7.4718 - learning_rate: 0.0010
Epoch 4/50
915/915 ━━━━━━━━━━━━━━━━━━━━ 0s 166us/step - accuracy: 0.5000 - loss: 2.1178 - val_accuracy: 0.0906 - val_loss: 7.2335 - learning_rate: 0.0010
Epoch 5/50
915/915 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.3567 - loss: 2.3558 - val_accuracy: 0.2412 - val_loss: 3.1690 - learning_rate: 0.0010
Epoch 6/50
915/915 ━━━━━━━━━━━━━━━━━━━━ 0s 173us/step - accuracy: 0.5667 - loss: 1.9313 - val_accuracy: 0.2496 - val_loss: 3.1776 - learning_rate: 0.0010
Epoch 7/50
915/915 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.3763 - loss: 2.2212 - val_accuracy: 0.0982 - val_loss: 5.0094 - learning_rate: 0.0010
Epoch 8/50
915/915 ━━━━━━━━━━━━━━━━━━━━ 0s 177us/step - accuracy: 0.4667 - loss: 2.0363 - val_accuracy: 0.1135 - val_loss: 4.3714 - learning_rate: 0.0010
Epoch 9/50
915/915 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.4357 - loss: 2.0056 - val_a

In [16]:
cnn_histories = return_best_model("CNN", cnn_models, X_train, y_train_enc, 30)

Model for CNN: <Sequential name=sequential_5, built=True>
Epoch 1/50
915/915 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.5362 - loss: 1.5724 - val_accuracy: 0.8302 - val_loss: 0.4911 - learning_rate: 0.0010
Epoch 2/50
915/915 ━━━━━━━━━━━━━━━━━━━━ 0s 261us/step - accuracy: 0.9333 - loss: 0.3886 - val_accuracy: 0.7719 - val_loss: 0.6644 - learning_rate: 0.0010
Epoch 3/50
915/915 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.9105 - loss: 0.3066 - val_accuracy: 0.8740 - val_loss: 0.3572 - learning_rate: 0.0010
Epoch 4/50
915/915 ━━━━━━━━━━━━━━━━━━━━ 0s 251us/step - accuracy: 1.0000 - loss: 0.0728 - val_accuracy: 0.8614 - val_loss: 0.3886 - learning_rate: 0.0010
Epoch 5/50
915/915 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.9493 - loss: 0.1733 - val_accuracy: 0.9752 - val_loss: 0.0797 - learning_rate: 0.0010
Epoch 6/50
915/915 ━━━━━━━━━━━━━━━━━━━━ 0s 224us/step - accuracy: 0.9000 - loss: 0.2694 - val_accuracy: 0.9777 - val_loss: 0.0781 - learning_rate: 0.0010
Epoch 7/50
915/915 ━━━━━

In [17]:
# Combine all histories and models
all_histories = dense_histories + cnn_histories

# Select the best model based on validation accuracy
best_model = max(all_histories, key=lambda x: x[0])[1]


# Evaluate the best model on the final test set
final_eval = best_model.evaluate(X_val, y_val_enc, verbose=0)
print(f"Best Model: {best_model} - Final Test Accuracy: {final_eval[1]}")


Best Model: <Sequential name=sequential_5, built=True> - Final Test Accuracy: 1.0


In [45]:
evaluate_model("CNN", best_model, X_val, y_val_enc)

113/113 - 0s - 2ms/step - accuracy: 0.9997 - loss: 0.0077
CNN Model - Test Accuracy: 0.999721

113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Unbiased Median Accuracy: 1.000000
Letter with Highest Accuracy: A
Letter with Lowest Accuracy: G

Top three errors:
-> Predict G as P
Only 1 errors detected

Overall Mean Accuracy: 0.999761

Letter A: Accuracy 1.000000
Letter B: Accuracy 1.000000
Letter C: Accuracy 1.000000
Letter D: Accuracy 1.000000
Letter E: Accuracy 1.000000
Letter F: Accuracy 1.000000
Letter G: Accuracy 0.994253
Letter H: Accuracy 1.000000
Letter I: Accuracy 1.000000
Letter K: Accuracy 1.000000
Letter L: Accuracy 1.000000
Letter M: Accuracy 1.000000
Letter N: Accuracy 1.000000
Letter O: Accuracy 1.000000
Letter P: Accuracy 1.000000
Letter Q: Accuracy 1.000000
Letter R: Accuracy 1.000000
Letter S: Accuracy 1.000000
Letter T: Accuracy 1.000000
Letter U: Accuracy 1.000000
Letter V: Accuracy 1.000000
Letter W: Accuracy 1.000000
Letter X: Accuracy 1.000000
Letter Y: Accuracy 1.000000
